In [5]:
from requests import get
import pandas as pd

url = 'https://www.imdb.com/list/ls055265443/'
response = get(url)

In [2]:
from bs4 import BeautifulSoup

html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [3]:
movie_cont = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_cont))
print(len(movie_cont))

<class 'bs4.element.ResultSet'>
50


In [4]:
names = []
year = []
genres = []
runtimes = []
metascores = []
imdb = []
audience = []
box = []

for movie in movie_cont:
    
    name = movie.h3.a.text
    names.append(name)
    
    year_ = movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
    year.append(year_)
    
    genre = movie.p.find('span', class_ = 'genre').text
    genres.append(genre)
    
    runtime = movie.p.find('span', class_ = 'runtime').text
    runtimes.append(runtime)
    
    viewer_rating = movie.p.span.text
    audience.append(viewer_rating)
    
    m_score = movie.find('span', class_ = 'metascore').text
    metascores.append(int(m_score))
    
    imdb_rating = float(movie.strong.text)
    imdb.append(imdb_rating)
    
    box_office = movie.find_all('span', attrs = {'name': 'nv'})[1].text
    box.append(box_office)

In [7]:
df = pd.DataFrame({'Name': names,
                  'Year': year,
                  'Genre': genres,
                  'Runtime': runtimes,
                  'Rating': audience,
                  'Metascore': metascores,
                  'IMDB': imdb,
                  'BoxOffice': box})

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
Name         50 non-null object
Year         50 non-null object
Genre        50 non-null object
Runtime      50 non-null object
Rating       50 non-null object
Metascore    50 non-null int64
IMDB         50 non-null float64
BoxOffice    50 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 3.2+ KB


In [9]:
df.head(50)

,Name,Year,Genre,Runtime,Rating,Metascore,IMDB,BoxOffice
0,Black Panther,(2018),"\nAction, Adventure, Sci-Fi",134 min,PG,88,7.3,$700.06M
1,Avengers: Infinity War,(2018),"\nAction, Adventure, Fantasy",149 min,PG,68,8.5,$678.82M
2,Incredibles 2,(2018),"\nAnimation, Action, Adventure",118 min,PG,80,7.7,$608.58M
3,Jurassic World: Fallen Kingdom,(2018),"\nAction, Adventure, Sci-Fi",128 min,PG,51,6.2,$417.72M
4,Aquaman,(2018),"\nAction, Adventure, Fantasy",143 min,PG,55,7.3,$334.04M
5,Deadpool 2,(2018),"\nAction, Adventure, Comedy",119 min,14A,66,7.8,$324.59M
6,The Grinch,(2018),"\nAnimation, Comedy, Family",86 min,G,51,6.3,$270.60M
7,Mission: Impossible - Fallout,(2018),"\nAction, Adventure, Thriller",147 min,PG,86,7.8,$220.16M
8,Ant-Man and the Wasp,(2018),"\nAction, Adventure, Comedy",118 min,PG,70,7.1,$216.65M
9,Bohemian Rhapsody,(2018),"\nBiography, Drama, Music",134 min,PG,49,8.1,$215.24M
